# Cornershop's Data Science Test

## Dev: Mateus Broilo
mateus.brilo90@gmail.com

* Objectives: Load Datasets and perform 

 # Summary

* Initials
    * Loading Libraries and Packages
    * Configurations
    * Global Variables
* Function's Definition
    * Class of Preprocess Funtions
* Loading data
* Data Preprocessing
    * Date Resources: day_of_week,  day_name, day_of_month, pure_time, day_of_month_RANGE
    * Transactions Resources: same_day_count,  same_day_valor_count, rank_same_day, diff_time
* Selecting Possible Useful Resources
* Saving preprocessed dataset  


# Initials
* Loading Libraries and Packages
* Configuration
* Global Variables

## Loading Libraries and Packages 

In [27]:
# general
import warnings
import time
import gc

#data
import pandas as pd
import numpy as np
from datetime import (datetime, date)

## visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
ls -l ../data/

total 504
-rw-rw-r-- 1 broilo broilo 514550 mai 20 16:56 20220520_corrected_sheet1.csv


## Configurations

In [29]:
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

rc={'font.size': 14, 'axes.labelsize': 14, 'legend.fontsize': 12, 
    'axes.titlesize': 24, 'xtick.labelsize': 14, 'ytick.labelsize': 12}

sns.set(rc=rc)
sns.set_palette("tab10")
sns.set_style("darkgrid")#sns.set_style("whitegrid")

## Global Variables

In [30]:
PATH_DATA = "../data/"
DATA = "20220520_corrected_sheet1.csv"
DATE = time.strftime("%Y%m%d")

# Function's Definition

## Class of Preprocess Funtions

In [31]:
class preprocessToolKit:
    
    def dropDuplicates(dataset):
        dataset = dataset.drop_duplicates(keep='first').reset_index(drop=True).copy()    
        return dataset
    
    def checkTypo(dataset):
        dataset['Data'] = dataset.Dia.astype(str) + ' ' + dataset.Hora.astype(str)
        uniqueDataLen = []
        for i in range(len(dataset.Data)):
            uniqueDataLen.append(len(dataset.Data[i]))
        uniqueDataLen = list(set(uniqueDataLen))
        idx = []
        for i in range(len(dataset.Data)):
            if len(dataset.Data[i]) > 19 or len(dataset.Data[i]) < 19:
                idx.append(dataset[dataset.Data == dataset.Data[i]].index.values[0])
            else:
                None
        dataset.drop(index=idx, axis=0, inplace=True)
        return dataset
    
    def dataTimeConverter(dataset, col1, col2):
        
        # day_of_week
        dataset['day_of_week'] = list(pd.to_datetime(dataset[col1], format='%Y-%m-%d').dt.dayofweek)
        dataset['day_name'] = list(pd.to_datetime(dataset[col1], format='%Y-%m-%d').dt.day_name())#.dayofweek)

        # day_of_month
        dataset[[col1+'_Ano', col1+'_Mês', col1+'_Dia']] = dataset[col1].str.split("-", expand=True)
        dataset.rename(columns={col1+'_Dia':'day_of_month'}, inplace=True)
        dataset['day_of_month'] = dataset['day_of_month'].astype(int)

        # pure_time
        dataset[[col2+'_HH', col2+'_MIN', col2+'_SEC']] = dataset[col2].str.split(":", expand=True)
        dataset['pure_time'] = dataset[col2+'_HH'].astype(int) + (dataset[col2+'_MIN'].astype(int) / 60) + (dataset[col2+'_SEC'].astype(float) / 3600)

        del dataset[col1+'_Ano'], dataset[col1+'_Mês'], dataset[col2+'_HH'], dataset[col2+'_MIN'], dataset[col2+'_SEC']
        
        conditions = [
            (dataset.day_of_month <= 5),
            ((dataset.day_of_month > 5) & (dataset.day_of_month <= 10)),
            ((dataset.day_of_month > 10) & (dataset.day_of_month <= 15)),
            ((dataset.day_of_month > 15) & (dataset.day_of_month <= 20)),
            ((dataset.day_of_month > 20) & (dataset.day_of_month <= 25)),
            (dataset.day_of_month > 25),
        ]

        choices = ['1-5', '6-10', '11-15', '16-20', '21-25', '>25']

        dataset['day_of_month_RANGE'] = np.select(conditions, choices, default='Tie')
        return dataset
    
    def recorrenciaTransacaoDiaria(dataset, col1, col2):
        dataset_sameDay = dataset[['Cartão','Dia']].value_counts().to_frame().reset_index()
        dataset_sameDay.rename(columns={0:'same_day_count'}, inplace=True)
        return dataset_sameDay
    
    def recorrenciaTransacaoDiariaValor(dataset, col1, col2, col3):
        dataset_sameDayValor = dataset[['Cartão','Dia','Valor']].value_counts().to_frame().reset_index()
        dataset_sameDayValor.rename(columns={0:'same_day_valor_count'}, inplace=True)
        return dataset_sameDayValor
        
    def merges(dataset, col1, col2, col3):
        dataset_sameDay = preprocessToolKit.recorrenciaTransacaoDiaria(dataset, col1, col2)
        dataset_sameDayValor = preprocessToolKit.recorrenciaTransacaoDiariaValor(dataset, col1, col2, col3)
        dataseta = dataset.merge(
            dataset_sameDay[[col1, col2, 'same_day_count']], 
            how='left', 
            left_on = [col1, col2], 
            right_on = [col1, col2]
        )
        dataset = dataseta.merge(
            dataset_sameDayValor[[col1, col2, col3, 'same_day_valor_count']], 
            how='left', 
            left_on = [col1, col2, col3], 
            right_on = [col1, col2, col3]
        )
        dataset = dataset.sort_values(by=['Cartão','Dia','Hora']).reset_index(drop=True).copy()
        return dataset
    
    def rankTransacao(dataset):
        dataset['rank_same_day'] = dataset.groupby(by=['Cartão','Dia'])['pure_time'].rank(method='first')
        return dataset
    
    def diffTime(dataset):
        diff_time = []
        for i in range(len(dataset)):
            if dataset.rank_same_day[i] == 1:
                diff_time.append(0)
            else:
                diff_time.append(dataset.pure_time[i] - dataset.pure_time[i-1])
        dataset['diff_time'] = diff_time
        return dataset

# Loading Datasets

In [32]:
df = pd.read_csv(PATH_DATA + DATA, sep=',')
print(df.shape)
df

(11128, 5)


,Dia,Hora,Valor,Cartão,CBK
0,2015-05-01,00:01:54,36.54,536518******2108,Não
1,2015-05-01,00:03:46,36.54,536518******2108,Não
2,2015-05-01,00:08:50,69.00,453211******1239,Não
3,2015-05-01,00:27:00,193.43,548827******1705,Não
4,2015-05-01,01:32:46,132.00,531681******9778,Não
...,...,...,...,...,...
11123,2015-05-30,23:07:01,53.00,514868******7409,Não
11124,2015-05-30,23:08:47,15.00,439354******5281,Não
11125,2015-05-30,23:15:24,20.00,549167******1648,Não
11126,2015-05-30,23:17:41,70.00,518759******8384,Não


# Data Preprocessing
* Drop Duplicates
* Check for typos
* Feature Engineering
    * Date Resources: day_of_week,  day_name, day_of_month, pure_time, day_of_month_RANGE
    * Transactions Resources: same_day_count,  same_day_valor_count, rank_same_day, diff_time


In [33]:
df_prep = df.copy()

## Drop Duplicates

In [34]:
df_prep = preprocessToolKit.dropDuplicates(df_prep)
df_prep

,Dia,Hora,Valor,Cartão,CBK
0,2015-05-01,00:01:54,36.54,536518******2108,Não
1,2015-05-01,00:03:46,36.54,536518******2108,Não
2,2015-05-01,00:08:50,69.00,453211******1239,Não
3,2015-05-01,00:27:00,193.43,548827******1705,Não
4,2015-05-01,01:32:46,132.00,531681******9778,Não
...,...,...,...,...,...
10999,2015-05-30,23:08:47,15.00,439354******5281,Não
11000,2015-05-30,23:15:24,20.00,549167******1648,Não
11001,2015-05-30,23:17:41,70.00,518759******8384,Não
11002,2015-05-30,23:51:31,20.00,518759******0329,Não


## Check for typos

In [35]:
df_prep = preprocessToolKit.checkTypo(df_prep)
df_prep

,Dia,Hora,Valor,Cartão,CBK,Data
0,2015-05-01,00:01:54,36.54,536518******2108,Não,2015-05-01 00:01:54
1,2015-05-01,00:03:46,36.54,536518******2108,Não,2015-05-01 00:03:46
2,2015-05-01,00:08:50,69.00,453211******1239,Não,2015-05-01 00:08:50
3,2015-05-01,00:27:00,193.43,548827******1705,Não,2015-05-01 00:27:00
4,2015-05-01,01:32:46,132.00,531681******9778,Não,2015-05-01 01:32:46
...,...,...,...,...,...,...
10999,2015-05-30,23:08:47,15.00,439354******5281,Não,2015-05-30 23:08:47
11000,2015-05-30,23:15:24,20.00,549167******1648,Não,2015-05-30 23:15:24
11001,2015-05-30,23:17:41,70.00,518759******8384,Não,2015-05-30 23:17:41
11002,2015-05-30,23:51:31,20.00,518759******0329,Não,2015-05-30 23:51:31


## Feature Engineering

### Date Resources: day_of_week,  day_name, day_of_month, pure_time, day_of_month_RANGE

In [36]:
df_prep = preprocessToolKit.dataTimeConverter(df_prep, 'Dia', 'Hora')
df_prep

,Dia,Hora,Valor,Cartão,CBK,Data,day_of_week,day_name,day_of_month,pure_time,day_of_month_RANGE
0,2015-05-01,00:01:54,36.54,536518******2108,Não,2015-05-01 00:01:54,4,Friday,1,0.031667,1-5
1,2015-05-01,00:03:46,36.54,536518******2108,Não,2015-05-01 00:03:46,4,Friday,1,0.062778,1-5
2,2015-05-01,00:08:50,69.00,453211******1239,Não,2015-05-01 00:08:50,4,Friday,1,0.147222,1-5
3,2015-05-01,00:27:00,193.43,548827******1705,Não,2015-05-01 00:27:00,4,Friday,1,0.450000,1-5
4,2015-05-01,01:32:46,132.00,531681******9778,Não,2015-05-01 01:32:46,4,Friday,1,1.546111,1-5
...,...,...,...,...,...,...,...,...,...,...,...
10999,2015-05-30,23:08:47,15.00,439354******5281,Não,2015-05-30 23:08:47,5,Saturday,30,23.146389,>25
11000,2015-05-30,23:15:24,20.00,549167******1648,Não,2015-05-30 23:15:24,5,Saturday,30,23.256667,>25
11001,2015-05-30,23:17:41,70.00,518759******8384,Não,2015-05-30 23:17:41,5,Saturday,30,23.294722,>25
11002,2015-05-30,23:51:31,20.00,518759******0329,Não,2015-05-30 23:51:31,5,Saturday,30,23.858611,>25


### Transactions Resources: same_day_count,  same_day_valor_count, rank_same_day, diff_time

In [37]:
df_prep = preprocessToolKit.merges(df_prep, 'Cartão','Dia','Valor')
df_prep

,Dia,Hora,Valor,Cartão,CBK,Data,day_of_week,day_name,day_of_month,pure_time,day_of_month_RANGE,same_day_count,same_day_valor_count
0,2015-05-06,09:37:46,198.0,400217******1137,Não,2015-05-06 09:37:46,2,Wednesday,6,9.629444,6-10,1,1
1,2015-05-27,23:37:20,172.5,400217******1353,Sim,2015-05-27 23:37:20,2,Wednesday,27,23.622222,>25,8,8
2,2015-05-27,23:38:58,172.5,400217******1353,Sim,2015-05-27 23:38:58,2,Wednesday,27,23.649444,>25,8,8
3,2015-05-27,23:40:15,172.5,400217******1353,Sim,2015-05-27 23:40:15,2,Wednesday,27,23.670833,>25,8,8
4,2015-05-27,23:41:38,172.5,400217******1353,Sim,2015-05-27 23:41:38,2,Wednesday,27,23.693889,>25,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10998,2015-05-20,22:52:46,112.0,554932******8129,Não,2015-05-20 22:52:46,2,Wednesday,20,22.879444,16-20,5,4
10999,2015-05-20,23:04:29,134.4,554932******8129,Não,2015-05-20 23:04:29,2,Wednesday,20,23.074722,16-20,5,1
11000,2015-05-06,18:15:53,207.0,554932******8142,Não,2015-05-06 18:15:53,2,Wednesday,6,18.264722,6-10,1,1
11001,2015-05-28,15:31:22,138.0,554932******9115,Não,2015-05-28 15:31:22,3,Thursday,28,15.522778,>25,1,1


In [38]:
df_prep = preprocessToolKit.rankTransacao(df_prep)
df_prep

,Dia,Hora,Valor,Cartão,CBK,Data,day_of_week,day_name,day_of_month,pure_time,day_of_month_RANGE,same_day_count,same_day_valor_count,rank_same_day
0,2015-05-06,09:37:46,198.0,400217******1137,Não,2015-05-06 09:37:46,2,Wednesday,6,9.629444,6-10,1,1,1.0
1,2015-05-27,23:37:20,172.5,400217******1353,Sim,2015-05-27 23:37:20,2,Wednesday,27,23.622222,>25,8,8,1.0
2,2015-05-27,23:38:58,172.5,400217******1353,Sim,2015-05-27 23:38:58,2,Wednesday,27,23.649444,>25,8,8,2.0
3,2015-05-27,23:40:15,172.5,400217******1353,Sim,2015-05-27 23:40:15,2,Wednesday,27,23.670833,>25,8,8,3.0
4,2015-05-27,23:41:38,172.5,400217******1353,Sim,2015-05-27 23:41:38,2,Wednesday,27,23.693889,>25,8,8,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10998,2015-05-20,22:52:46,112.0,554932******8129,Não,2015-05-20 22:52:46,2,Wednesday,20,22.879444,16-20,5,4,4.0
10999,2015-05-20,23:04:29,134.4,554932******8129,Não,2015-05-20 23:04:29,2,Wednesday,20,23.074722,16-20,5,1,5.0
11000,2015-05-06,18:15:53,207.0,554932******8142,Não,2015-05-06 18:15:53,2,Wednesday,6,18.264722,6-10,1,1,1.0
11001,2015-05-28,15:31:22,138.0,554932******9115,Não,2015-05-28 15:31:22,3,Thursday,28,15.522778,>25,1,1,1.0


In [39]:
df_prep = preprocessToolKit.diffTime(df_prep)
df_prep

,Dia,Hora,Valor,Cartão,CBK,Data,day_of_week,day_name,day_of_month,pure_time,day_of_month_RANGE,same_day_count,same_day_valor_count,rank_same_day,diff_time
0,2015-05-06,09:37:46,198.0,400217******1137,Não,2015-05-06 09:37:46,2,Wednesday,6,9.629444,6-10,1,1,1.0,0.000000
1,2015-05-27,23:37:20,172.5,400217******1353,Sim,2015-05-27 23:37:20,2,Wednesday,27,23.622222,>25,8,8,1.0,0.000000
2,2015-05-27,23:38:58,172.5,400217******1353,Sim,2015-05-27 23:38:58,2,Wednesday,27,23.649444,>25,8,8,2.0,0.027222
3,2015-05-27,23:40:15,172.5,400217******1353,Sim,2015-05-27 23:40:15,2,Wednesday,27,23.670833,>25,8,8,3.0,0.021389
4,2015-05-27,23:41:38,172.5,400217******1353,Sim,2015-05-27 23:41:38,2,Wednesday,27,23.693889,>25,8,8,4.0,0.023056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10998,2015-05-20,22:52:46,112.0,554932******8129,Não,2015-05-20 22:52:46,2,Wednesday,20,22.879444,16-20,5,4,4.0,0.022222
10999,2015-05-20,23:04:29,134.4,554932******8129,Não,2015-05-20 23:04:29,2,Wednesday,20,23.074722,16-20,5,1,5.0,0.195278
11000,2015-05-06,18:15:53,207.0,554932******8142,Não,2015-05-06 18:15:53,2,Wednesday,6,18.264722,6-10,1,1,1.0,0.000000
11001,2015-05-28,15:31:22,138.0,554932******9115,Não,2015-05-28 15:31:22,3,Thursday,28,15.522778,>25,1,1,1.0,0.000000


## Encoding

In [40]:
df_prep = pd.get_dummies(df_prep, columns=['day_name', 'day_of_month_RANGE','CBK'])
df_prep

,Dia,Hora,Valor,Cartão,Data,day_of_week,day_of_month,pure_time,same_day_count,same_day_valor_count,...,day_name_Tuesday,day_name_Wednesday,day_of_month_RANGE_1-5,day_of_month_RANGE_11-15,day_of_month_RANGE_16-20,day_of_month_RANGE_21-25,day_of_month_RANGE_6-10,day_of_month_RANGE_>25,CBK_Não,CBK_Sim
0,2015-05-06,09:37:46,198.0,400217******1137,2015-05-06 09:37:46,2,6,9.629444,1,1,...,0,1,0,0,0,0,1,0,1,0
1,2015-05-27,23:37:20,172.5,400217******1353,2015-05-27 23:37:20,2,27,23.622222,8,8,...,0,1,0,0,0,0,0,1,0,1
2,2015-05-27,23:38:58,172.5,400217******1353,2015-05-27 23:38:58,2,27,23.649444,8,8,...,0,1,0,0,0,0,0,1,0,1
3,2015-05-27,23:40:15,172.5,400217******1353,2015-05-27 23:40:15,2,27,23.670833,8,8,...,0,1,0,0,0,0,0,1,0,1
4,2015-05-27,23:41:38,172.5,400217******1353,2015-05-27 23:41:38,2,27,23.693889,8,8,...,0,1,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10998,2015-05-20,22:52:46,112.0,554932******8129,2015-05-20 22:52:46,2,20,22.879444,5,4,...,0,1,0,0,1,0,0,0,1,0
10999,2015-05-20,23:04:29,134.4,554932******8129,2015-05-20 23:04:29,2,20,23.074722,5,1,...,0,1,0,0,1,0,0,0,1,0
11000,2015-05-06,18:15:53,207.0,554932******8142,2015-05-06 18:15:53,2,6,18.264722,1,1,...,0,1,0,0,0,0,1,0,1,0
11001,2015-05-28,15:31:22,138.0,554932******9115,2015-05-28 15:31:22,3,28,15.522778,1,1,...,0,0,0,0,0,0,0,1,1,0


# Selecting Possible Useful Resources

In [41]:
list(df_prep)

['Dia',
 'Hora',
 'Valor',
 'Cartão',
 'Data',
 'day_of_week',
 'day_of_month',
 'pure_time',
 'same_day_count',
 'same_day_valor_count',
 'rank_same_day',
 'diff_time',
 'day_name_Friday',
 'day_name_Monday',
 'day_name_Saturday',
 'day_name_Sunday',
 'day_name_Thursday',
 'day_name_Tuesday',
 'day_name_Wednesday',
 'day_of_month_RANGE_1-5',
 'day_of_month_RANGE_11-15',
 'day_of_month_RANGE_16-20',
 'day_of_month_RANGE_21-25',
 'day_of_month_RANGE_6-10',
 'day_of_month_RANGE_>25',
 'CBK_Não',
 'CBK_Sim']

In [42]:
selected = [
    #'Dia',
    #'Hora',
    'Valor',
    'Cartão', #-> index!
    #'CBK', #-> TARGET!
    'Data',
    #'day_of_week',
    #'day_name',
    #'day_of_month',
    'pure_time',
    #'day_of_month_RANGE',
    'same_day_count',
    'same_day_valor_count',
    'rank_same_day',
    'diff_time',
    'day_name_Friday',
    'day_name_Monday',
    'day_name_Saturday',
    'day_name_Sunday',
    'day_name_Thursday',
    'day_name_Tuesday',
    'day_name_Wednesday',
    'day_of_month_RANGE_1-5',
    'day_of_month_RANGE_11-15',
    'day_of_month_RANGE_16-20',
    'day_of_month_RANGE_21-25',
    'day_of_month_RANGE_6-10',
    'day_of_month_RANGE_>25',
    #'CBK_Não',
    'CBK_Sim' #-> TARGET!
]

In [43]:
df_prep[selected]

,Valor,Cartão,Data,pure_time,same_day_count,same_day_valor_count,rank_same_day,diff_time,day_name_Friday,day_name_Monday,...,day_name_Thursday,day_name_Tuesday,day_name_Wednesday,day_of_month_RANGE_1-5,day_of_month_RANGE_11-15,day_of_month_RANGE_16-20,day_of_month_RANGE_21-25,day_of_month_RANGE_6-10,day_of_month_RANGE_>25,CBK_Sim
0,198.0,400217******1137,2015-05-06 09:37:46,9.629444,1,1,1.0,0.000000,0,0,...,0,0,1,0,0,0,0,1,0,0
1,172.5,400217******1353,2015-05-27 23:37:20,23.622222,8,8,1.0,0.000000,0,0,...,0,0,1,0,0,0,0,0,1,1
2,172.5,400217******1353,2015-05-27 23:38:58,23.649444,8,8,2.0,0.027222,0,0,...,0,0,1,0,0,0,0,0,1,1
3,172.5,400217******1353,2015-05-27 23:40:15,23.670833,8,8,3.0,0.021389,0,0,...,0,0,1,0,0,0,0,0,1,1
4,172.5,400217******1353,2015-05-27 23:41:38,23.693889,8,8,4.0,0.023056,0,0,...,0,0,1,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10998,112.0,554932******8129,2015-05-20 22:52:46,22.879444,5,4,4.0,0.022222,0,0,...,0,0,1,0,0,1,0,0,0,0
10999,134.4,554932******8129,2015-05-20 23:04:29,23.074722,5,1,5.0,0.195278,0,0,...,0,0,1,0,0,1,0,0,0,0
11000,207.0,554932******8142,2015-05-06 18:15:53,18.264722,1,1,1.0,0.000000,0,0,...,0,0,1,0,0,0,0,1,0,0
11001,138.0,554932******9115,2015-05-28 15:31:22,15.522778,1,1,1.0,0.000000,0,0,...,1,0,0,0,0,0,0,0,1,0


# Saving preprocessed dataset

In [44]:
df_prep[selected].to_csv(PATH_DATA + DATE + '_data_prep.csv',
             sep=',',
             index=False)